<a href="https://colab.research.google.com/github/aluposto/video_codec_test_11/blob/main/4rates_Copy_of_dcvc_Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, os, platform
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi
!python --version


CUDA available: True
Mon Sep  1 12:58:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [2]:
!rm -rf DCVC
!git clone https://github.com/microsoft/DCVC.git
%cd DCVC

# Install build tools
!sudo apt-get update -y
!sudo apt-get install -y cmake g++ ninja-build

# Install PyTorch 2.6 with CUDA 12.6 (fallback to CUDA 12.1 if needed)
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126 || \
 pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Python dependencies
!pip install -r requirements.txt

# Build C++ bitstream writer
%cd src/cpp
!pip install .
# Build CUDA fused inference kernels
%cd ../layers/extensions/inference
!pip install .

# Back to root
%cd ../../../..


Cloning into 'DCVC'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 453 (delta 75), reused 43 (delta 43), pack-reused 326 (from 2)
Receiving objects: 100% (453/453), 7.43 MiB | 15.53 MiB/s, done.
Resolving deltas: 100% (184/184), done.
/content/DCVC
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.2 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archi

In [3]:
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Torch: 2.8.0+cu126
CUDA available: True
GPU name: Tesla T4


In [4]:
!pip install tqdm requests


In [6]:
import requests
from tqdm import tqdm

url = "https://ultravideo.fi/video/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z"
output_path = "/media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z"

# Ensure folder exists
import os
os.makedirs("/media/data/UVG", exist_ok=True)

# Stream download with progress bar
response = requests.get(url, stream=True)
total_size = int(response.headers.get("content-length", 0))
block_size = 1024 * 1024  # 1 MB chunks
t = tqdm(total=total_size, unit="B", unit_scale=True, desc="Downloading")

with open(output_path, "wb") as f:
    for data in response.iter_content(block_size):
        f.write(data)
        t.update(len(data))
t.close()

print("✅ Download finished:", output_path)



Downloading:   0%|          | 0.00/925M [00:23<?, ?B/s]

Downloading:   0%|          | 1.05M/925M [00:01<21:18, 723kB/s]
Downloading:   0%|          | 2.10M/925M [00:01<10:51, 1.42MB/s]
Downloading:   0%|          | 4.19M/925M [00:01<05:09, 2.98MB/s]
Downloading:   1%|          | 7.34M/925M [00:02<02:51, 5.34MB/s]
Downloading:   1%|          | 11.5M/925M [00:02<01:50, 8.27MB/s]
Downloading:   2%|▏         | 15.7M/925M [00:02<01:26, 10.6MB/s]
Downloading:   2%|▏         | 19.9M/925M [00:02<01:13, 12.3MB/s]
Downloading:   3%|▎         | 24.1M/925M [00:03<01:06, 13.6MB/s]
Downloading:   3%|▎         | 27.3M/925M [00:03<01:07, 13.3MB/s]
Downloading:   3%|▎         | 31.5M/925M [00:03<01:02, 14.3MB/s]
Downloading:   4%|▍         | 35.7M/925M [00:03<00:59, 15.1MB/s]
Downloading:   4%|▍         | 39.8M/925M [00:04<00:56, 15.6MB/s]
Downloading:   5%|▍         | 44.0M/925M [00:04<00:55, 15.9MB/s]
Downloading:   5%|▌         | 48.2M/925M [00:04<00:54, 16.2MB/s]
Downloading:   6%|▌         | 51.

✅ Download finished: /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z


In [11]:
!sudo apt-get install -y p7zip-full
!7z x /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z -o/media/data/UVG/
!ls -lh /media/data/UVG


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /media/data/UVG/                          1 file, 925430047 bytes (883 MiB)

Extracting archive: /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
--
Path = /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
Type = 7z
Physical Size = 925430047
Headers Size = 246
Method = LZMA:16 BZip2
Solid = -
Blocks = 2

  0%      0% - Beauty_1920x1080_120fps_420_8bit_YUV.yuv                                               

In [12]:
!ls -lh /media/data/UVG


total 2.6G
-rw-r--r-- 1 root root 883M Sep  1 13:05 Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
-rw-r--r-- 1 root root 1.8G Oct  7  2013 Beauty_1920x1080_120fps_420_8bit_YUV.yuv
-rw-r--r-- 1 root root  249 Oct 24  2013 Beauty_Copyright.txt


In [14]:
%cd /content/DCVC

MODEL_I="./checkpoints/cvpr2025_image.pth.tar"
MODEL_P="./checkpoints/cvpr2025_video.pth.tar"
CONFIG="./dataset_config_example_yuv420.json"  # expects /media/data/HEVC_B
OUTJSON="output.json"

!python test_video.py \
  --model_path_i "$MODEL_I" \
  --model_path_p "$MODEL_P" \
  --rate_num 2 \
  --test_config "$CONFIG" \
  --cuda 1 \
  -w 1 \
  --write_stream 1 \
  --force_zero_thres 0.12 \
  --output_path "$OUTJSON" \
  --force_intra_period -1 \
  --reset_interval 64 \
  --force_frame_num -1 \
  --check_existing 0 \
  --verbose 1


/content/DCVC
testing 2 rates, using qp: 0, 63, 
  0% 0/106 [00:00<?, ?it/s]created folder: out_bin/UVG
encoding/decoding 600 frames, average encoding time 44.886 ms, average decoding time 52.355 ms.
  1% 1/106 [02:49<4:56:01, 169.16s/it]encoding/decoding 600 frames, average encoding time 46.708 ms, average decoding time 53.910 ms.
  2% 2/106 [05:32<4:47:54, 166.10s/it]
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.12/concurrent/futures/process.py", line 264, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 370, in worker
    result = run_one_point_with_stream(p_frame_net, i_frame_net, args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 147, in run_one_point_with_stream
    src_reader = get_src_reader(args)
                 ^^^^^

In [15]:
%cd /content/DCVC

MODEL_I="./checkpoints/cvpr2025_image.pth.tar"
MODEL_P="./checkpoints/cvpr2025_video.pth.tar"
CONFIG="./dataset_config_example_yuv420.json"  # expects /media/data/HEVC_B
OUTJSON="output.json"

!python test_video.py \
  --model_path_i "$MODEL_I" \
  --model_path_p "$MODEL_P" \
  --rate_num 4 \
  --test_config "$CONFIG" \
  --cuda 1 \
  -w 1 \
  --write_stream 1 \
  --force_zero_thres 0.12 \
  --output_path "$OUTJSON" \
  --force_intra_period -1 \
  --reset_interval 64 \
  --force_frame_num -1 \
  --check_existing 0 \
  --verbose 1


/content/DCVC
testing 4 rates, using qp: 0, 21, 42, 63, 
  0% 0/212 [00:00<?, ?it/s]encoding/decoding 600 frames, average encoding time 44.734 ms, average decoding time 51.587 ms.
  0% 1/212 [02:45<9:42:29, 165.64s/it]encoding/decoding 600 frames, average encoding time 46.426 ms, average decoding time 54.205 ms.
  1% 2/212 [05:25<9:28:38, 162.47s/it]encoding/decoding 600 frames, average encoding time 46.414 ms, average decoding time 54.273 ms.
  1% 3/212 [08:07<9:23:46, 161.85s/it]encoding/decoding 600 frames, average encoding time 46.478 ms, average decoding time 54.457 ms.
  2% 4/212 [10:48<9:22:22, 162.23s/it]
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.12/concurrent/futures/process.py", line 264, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 370, in worker
    result = run_one_point_with